In [1]:
# Available flags:	
# d	definition differs (see metadata)
# e	estimated
# s	Eurostat estimate

In [2]:
import pandas as pd
import numpy as np

file_path = 'data/estat_env_wasfw.tsv'

# Read Data

In [3]:
df = pd.read_csv(file_path)

# FIX for 'geo\TIME_PERIOD\t2020 \t2021 ' column
values = df['geo\TIME_PERIOD\t2020 \t2021 '].to_list()
new_list = []
for i in values:
    new_list.append(i.split('\t'))
df2 = pd.DataFrame(new_list, columns=['geo', '2020', '2021']) # columns separated with tab (\t)

# Merge normal and fixed parts of the data
df = pd.concat([df, df2], axis=1)
df = df.drop(columns = ['geo\TIME_PERIOD\t2020 \t2021 '])
df

,freq,wst_oper,waste,nace_r2,unit,geo,2020,2021
0,A,COL,W091_092_101_FD,A01_A03_FOOD,KG_HAB,AT,2,1
1,A,COL,W091_092_101_FD,A01_A03_FOOD,KG_HAB,BE,3,3
2,A,COL,W091_092_101_FD,A01_A03_FOOD,KG_HAB,BG,9,9
3,A,COL,W091_092_101_FD,A01_A03_FOOD,KG_HAB,CY,49 d,:
4,A,COL,W091_092_101_FD,A01_A03_FOOD,KG_HAB,CZ,3,:
...,...,...,...,...,...,...,...,...
331,A,COL,W091_092_101_FD,TOT,T,PL,4246787,4281212 e
332,A,COL,W091_092_101_FD,TOT,T,PT,1812307 e,1874832 e
333,A,COL,W091_092_101_FD,TOT,T,SE,925000 e,900000 e
334,A,COL,W091_092_101_FD,TOT,T,SI,143570,143254


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336 entries, 0 to 335
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   freq      336 non-null    object
 1   wst_oper  336 non-null    object
 2   waste     336 non-null    object
 3   nace_r2   336 non-null    object
 4   unit      336 non-null    object
 5   geo       336 non-null    object
 6   2020      336 non-null    object
 7   2021      336 non-null    object
dtypes: object(8)
memory usage: 21.1+ KB


# Preprocessing

In [5]:
def pivot_frame(df):
    renamings = {
        'TOT' : 'total',
        'I55_I56_N-S_FOOD' : 'restaurants',
        'G46_G47_FOOD' : 'retail',
        'C10_C11' : 'manufacture',
        'A01_A03_FOOD' : 'primary_prod',
        'HH':'housholds'
    }
    columns_order = [
        'total',
        'primary_prod',
        'manufacture',
        'retail',
        'restaurants',
        'housholds',
    ]

    df_y2020 = df.pivot(index='geo', columns='nace_r2', values='2020')
    df_y2021 = df.pivot(index='geo', columns='nace_r2', values='2021')

    df_y2020 = df_y2020.rename(columns = renamings)[columns_order]
    df_y2021 = df_y2021.rename(columns = renamings)[columns_order]

    df_y2020['year'] = 2020
    df_y2021['year'] = 2021

    df = pd.concat([df_y2020, df_y2021]).reset_index()
    return df

def preprocess(df):
    columns = [
        'total',
        'primary_prod',
        'manufacture',
        'retail',
        'restaurants',
        'housholds',
    ]
    
    for col in columns:
        df[col] = df[col].apply(lambda x: int(x.split()[0]) if ':' not in x else np.nan)
    return df

## KGs per Capita

In [6]:
kg_hab = pivot_frame(df[(df['unit'] == 'KG_HAB')])
kg_hab = preprocess(kg_hab)
kg_hab

nace_r2,geo,total,primary_prod,manufacture,retail,restaurants,housholds,year
0,AT,136.0,2.0,19.0,9.0,23.0,83.0,2020
1,BE,250.0,3.0,161.0,6.0,8.0,71.0,2020
2,BG,108.0,9.0,19.0,7.0,18.0,55.0,2020
3,CY,397.0,49.0,190.0,56.0,30.0,71.0,2020
4,CZ,91.0,3.0,9.0,6.0,4.0,69.0,2020
5,DE,131.0,2.0,19.0,9.0,22.0,78.0,2020
6,DK,221.0,11.0,102.0,17.0,11.0,79.0,2020
7,EE,125.0,18.0,24.0,15.0,8.0,61.0,2020
8,EL,191.0,35.0,35.0,14.0,21.0,87.0,2020
9,ES,90.0,18.0,30.0,7.0,4.0,30.0,2020


## Tonnes

In [7]:
tonne = pivot_frame(df[(df['unit'] == 'T')])
tonne = preprocess(tonne)
tonne

nace_r2,geo,total,primary_prod,manufacture,retail,restaurants,housholds,year
0,AT,1211534.0,13879.0,173734.0,84326.0,201956.0,737639.0,2020
1,BE,2881897.0,38699.0,1862177.0,73591.0,88333.0,819097.0,2020
2,BG,747941.0,63621.0,132817.0,46639.0,121867.0,382997.0,2020
3,CY,354021.0,43564.0,169706.0,50268.0,27145.0,63338.0,2020
4,CZ,972445.0,27022.0,100339.0,64394.0,37941.0,742749.0,2020
5,DE,10922321.0,190203.0,1612505.0,762352.0,1860980.0,6496282.0,2020
6,DK,1286488.0,66452.0,596599.0,99500.0,62544.0,461392.0,2020
7,EE,166513.0,23612.0,31622.0,19976.0,10739.0,80564.0,2020
8,EL,2048189.0,372204.0,375158.0,150472.0,220032.0,930323.0,2020
9,ES,4260845.0,845620.0,1419257.0,348219.0,213023.0,1434726.0,2020


In [8]:
# 2020 only
tonne[tonne['year']==2020]

nace_r2,geo,total,primary_prod,manufacture,retail,restaurants,housholds,year
0,AT,1211534.0,13879.0,173734.0,84326.0,201956.0,737639.0,2020
1,BE,2881897.0,38699.0,1862177.0,73591.0,88333.0,819097.0,2020
2,BG,747941.0,63621.0,132817.0,46639.0,121867.0,382997.0,2020
3,CY,354021.0,43564.0,169706.0,50268.0,27145.0,63338.0,2020
4,CZ,972445.0,27022.0,100339.0,64394.0,37941.0,742749.0,2020
5,DE,10922321.0,190203.0,1612505.0,762352.0,1860980.0,6496282.0,2020
6,DK,1286488.0,66452.0,596599.0,99500.0,62544.0,461392.0,2020
7,EE,166513.0,23612.0,31622.0,19976.0,10739.0,80564.0,2020
8,EL,2048189.0,372204.0,375158.0,150472.0,220032.0,930323.0,2020
9,ES,4260845.0,845620.0,1419257.0,348219.0,213023.0,1434726.0,2020


In [9]:
# Portugal only
tonne[tonne['geo']=='PT']

nace_r2,geo,total,primary_prod,manufacture,retail,restaurants,housholds,year
24,PT,1812307.0,101384.0,61719.0,214233.0,161399.0,1273572.0,2020
52,PT,1874832.0,126392.0,75257.0,224838.0,162903.0,1285442.0,2021


In [10]:
# Portugal only
kg_hab[kg_hab['geo']=='PT']

nace_r2,geo,total,primary_prod,manufacture,retail,restaurants,housholds,year
24,PT,176.0,10.0,6.0,21.0,16.0,124.0,2020
52,PT,181.0,12.0,7.0,22.0,16.0,124.0,2021


# Data Exploration

In [11]:
tonne.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   geo           56 non-null     object 
 1   total         50 non-null     float64
 2   primary_prod  50 non-null     float64
 3   manufacture   50 non-null     float64
 4   retail        50 non-null     float64
 5   restaurants   50 non-null     float64
 6   housholds     50 non-null     float64
 7   year          56 non-null     int64  
dtypes: float64(6), int64(1), object(1)
memory usage: 3.6+ KB


In [12]:
kg_hab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   geo           56 non-null     object 
 1   total         50 non-null     float64
 2   primary_prod  50 non-null     float64
 3   manufacture   50 non-null     float64
 4   retail        50 non-null     float64
 5   restaurants   50 non-null     float64
 6   housholds     50 non-null     float64
 7   year          56 non-null     int64  
dtypes: float64(6), int64(1), object(1)
memory usage: 3.6+ KB


In [13]:
tonne.describe()

nace_r2,total,primary_prod,manufacture,retail,restaurants,housholds,year
count,5.000000e+01,5.000000e+01,5.000000e+01,5.000000e+01,5.000000e+01,5.000000e+01,56.000000
mean,4.216434e+06,3.934308e+05,8.794753e+05,2.945726e+05,3.744214e+05,2.276534e+06,2020.500000
std,1.144418e+07,1.056832e+06,2.401725e+06,8.043943e+05,1.075450e+06,6.205370e+06,0.504525
min,7.958900e+04,7.100000e+01,4.668000e+03,3.910000e+03,4.495000e+03,4.672700e+04,2020.000000
25%,3.846600e+05,2.365000e+04,3.341525e+04,2.181750e+04,2.404825e+04,2.258740e+05,2020.000000
50%,8.917430e+05,5.404950e+04,1.588215e+05,6.939450e+04,8.568400e+04,4.841960e+05,2020.500000
75%,2.754936e+06,1.871418e+05,6.049195e+05,2.098050e+05,1.844515e+05,1.000362e+06,2021.000000
max,5.840000e+07,5.500000e+06,1.240000e+07,4.200000e+06,5.400000e+06,3.140000e+07,2021.000000


In [14]:
kg_hab.describe()

nace_r2,total,primary_prod,manufacture,retail,restaurants,housholds,year
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,56.000000
mean,138.800000,12.640000,32.440000,10.600000,13.040000,69.920000,2020.500000
std,56.764821,10.352482,42.228803,7.889544,9.745664,19.783357,0.504525
min,68.000000,0.000000,2.000000,1.000000,1.000000,30.000000,2020.000000
25%,106.500000,3.250000,9.000000,7.000000,5.000000,59.250000,2020.000000
50%,131.000000,11.000000,19.000000,9.000000,12.000000,67.500000,2020.500000
75%,147.750000,17.750000,29.000000,12.000000,17.750000,81.250000,2021.000000
max,397.000000,49.000000,190.000000,56.000000,45.000000,124.000000,2021.000000
